In [1]:
%load_ext autoreload

import xarray as xr
import rasterio
import pandas as pd
import numpy as np
import osr
import os
import arcpy
import datetime

sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
import arc, satfetcher, tools

sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import gdvspectra, cog

In [2]:
xr.open_dataset(r"C:\Users\Lewis\Desktop\test\test.nc")

<xarray.Dataset>
Dimensions:                    (albers_conical_equal_area: 0, x: 104, y: 121)
Coordinates:
  * x                          (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06
  * y                          (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06
  * albers_conical_equal_area  (albers_conical_equal_area) int32 
Data variables:
    mdr_thresh.crf             (y, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    Conventions:      CF-1.0
    Source_Software:  Esri ArcGIS

In [3]:
#ds_thresh = xr.open_dataset(r"C:\Users\Lewis\Desktop\working\thresh.nc")

In [4]:
arcpy.env.addOutputsToMap = False
mdr = arcpy.CopyRaster_management(in_raster=r"C:\Users\Lewis\Desktop\working\thresh.nc", 
                                  out_rasterdataset='mdr_thresh.crf')

In [69]:
mdr_filter = arcpy.sa.MajorityFilter(mdr, "FOUR", "MAJORITY")
mdr_filter.save('in_memory/mdr_thresh_filter.crf')

In [70]:
mdr_filter_binary = arcpy.sa.Con(mdr_filter, 1, None, "Value = 1")
mdr_filter_binary.save('in_memory/mdr_thresh_filter_binary.crf')

In [71]:
mdr_filter = arcpy.CopyRaster_management(in_raster='in_memory/mdr_thresh_filter_binary.crf', 
                                  out_rasterdataset='mdr_thresh_filter.crf')

In [72]:
# todo make this safe
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.activeMap
m.addDataFromPath(mdr_filter) 

In [89]:
# apply cmap
arcpy.env.addOutputsToMap = True
lyr = arc.apply_cmap(aprx=aprx, 
                     #lyr_name='mdr_thresh.crf', 
                     lyr_name='mdr_thresh_filter.crf', 
                     cmap_name='Set 2 (3 Classes)', 
                     cutoff_pct=0.01)

# set alpha
lyr.transparency = 25

In [33]:
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\working\like.nc")

In [37]:
da = ds.isel(time=0, drop=True)
da.to_array().to_dataset('variable')
#da.to_netcdf(r'C:\Users\Lewis\Desktop\working\test.nc')

<xarray.Dataset>
Dimensions:      (x: 104, y: 121)
Coordinates:
    spatial_ref  int32 3577
  * y            (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x            (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
Data variables:
    like         (y, x) float64 0.03219 0.02359 0.02164 ... 0.0004653 0.0002348
Attributes:
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.28823e+06  0.00000e+00 -...
    res:               30
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bbox:         [119.32132693 -22.77485339 119.34873642 -22.74501035]
    orig_slc_off:      False
    orig_resample:     Nearest

In [29]:
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\working\test.nc")
ds

<xarray.Dataset>
Dimensions:       (x: 104, y: 121)
Coordinates:
  * y             (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x             (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref   int32 3577
Data variables:
    nbart_blue    (y, x) int16 431 441 450 457 438 432 ... 351 340 357 331 342
    nbart_green   (y, x) int16 680 711 737 729 684 670 ... 578 565 594 584 614
    nbart_red     (y, x) int16 1076 1178 1236 1122 1028 ... 1060 1147 1142 1205
    nbart_nir     (y, x) int16 1559 1635 1678 1651 1560 ... 1410 1534 1529 1582
    nbart_swir_1  (y, x) int16 2186 2347 2455 2397 2121 ... 2479 2569 2716 2772
    nbart_swir_2  (y, x) int16 1729 1896 1950 1844 1632 ... 2177 2241 2309 2310
    oa_fmask      (y, x) int16 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
Attributes:
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.28823e+06  0.00000e+00 -...
    res:               30
    nodatavals:        -999
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bbox:         [119.32132693 -22.77485339 119.34873642 -22.74501035]
    orig_slc_off:      False
    orig_resample:     Nearest